In [2]:
from iroha import Iroha, IrohaCrypto, IrohaGrpc
from iroha.primitive_pb2 import *
import ed25519

net = IrohaGrpc('localhost:50051')
admin = Iroha("admin@test")


In [ ]:
def create_domain(domain):
    if domain == None:
        return None
    cmds = [
        admin.command("CreateDomain", domain_id=domain, default_role="admin")
    ]

    tx = admin.transaction(cmds,creator_account="admin@test")
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    return tx

In [16]:
def create_asset(asset):
    if asset == None:
        return None
    cmds = [
        admin.command("CreateAsset", asset_name=asset, domain_id="chain", precision=0)
    ]

    tx = admin.transaction(cmds)
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    return tx
create_asset("tinycoin")

('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)


payload {
  reduced_payload {
    commands {
      create_asset {
        asset_name: "tinycoin"
        domain_id: "chain"
      }
    }
    creator_account_id: "admin@test"
    created_time: 1646282999129
    quorum: 1
  }
}
signatures {
  public_key: "313a07e6384776ed95447710d15e59148473ccfc052a681317a72a69f2a49910"
  signature: "e157a7812c7a8cf02ab2660c6d86a12ad1a11299f09fb45e8a032714cc9b3d8a6ea9b01e132c0993ff8119f87f310e81fd3f084842c297209249f3988e908409"
}

In [21]:
def add_asset_quantity():
    cmds = [
        admin.command("AddAssetQuantity", asset_id="tinycoin#chain", amount="100")
    ]

    tx = admin.transaction(cmds)
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    return tx
add_asset_quantity()

('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)


payload {
  reduced_payload {
    commands {
      add_asset_quantity {
        asset_id: "tinycoin#chain"
        amount: "100"
      }
    }
    creator_account_id: "admin@test"
    created_time: 1646284036348
    quorum: 1
  }
}
signatures {
  public_key: "313a07e6384776ed95447710d15e59148473ccfc052a681317a72a69f2a49910"
  signature: "ce9956272bc8ec9add58c32f042a995e89d70ac2196ffb366f1bf6c20bdc621d984b5f7cc39691fdc88669dee77bc1b911d4f5130fa3ab52fb845e004139fa01"
}

In [30]:
def account_assets(account_id):
    query = admin.query('GetAccountAssets', account_id=account_id)
    IrohaCrypto.sign_query(query, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")

    response = net.send_query(query)
    data = response.account_assets_response.account_assets
    for asset in data:
        print('Asset id = {}, balance = {}'.format(
            asset.asset_id, asset.balance))
print("Account assets:")
account_assets("admin@test")
print("Bob's assets: ")
account_assets("bob@test")

Account assets:
Asset id = tinycoin#chain, balance = 90
Bob's assets: 
Asset id = tinycoin#chain, balance = 10


In [29]:
def transfer_assets(account_id, dest_account_id, asset_id, amount):
    cmds = [
        admin.command('TransferAsset', src_account_id=account_id, dest_account_id=dest_account_id, asset_id=asset_id, amount=amount)
    ]

    tx = admin.transaction(cmds)
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    return tx
transfer_assets("admin@test", "bob@test", "tinycoin#chain", "10")

('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)


payload {
  reduced_payload {
    commands {
      transfer_asset {
        src_account_id: "admin@test"
        dest_account_id: "bob@test"
        asset_id: "tinycoin#chain"
        amount: "10"
      }
    }
    creator_account_id: "admin@test"
    created_time: 1646284734980
    quorum: 1
  }
}
signatures {
  public_key: "313a07e6384776ed95447710d15e59148473ccfc052a681317a72a69f2a49910"
  signature: "cc531f305bb8e4e1d0d6ea222b2ecffc9ca85c289c58b649cfead8dece89652b16fc92f47cb9f0c22f0afe8e1c74afa82fe966a1845827f3a002ea2e9c22ea01"
}